In [1]:
import pandas as pd
import clickhouse_connect
import os

In [ ]:
client = clickhouse_connect.get_client(
    host='clickhouse',
    port=8123,  # HTTP interface
    username='abc',
    password='xyz'
)

In [3]:
client.command('DROP DATABASE IF EXISTS superficie_bd')

In [4]:
client.command("CREATE DATABASE IF NOT EXISTS superficie_bd") #creación de base de datos para bosques y cultivos 

In [5]:
print(client.query("SHOW DATABASES").result_rows)

[('INFORMATION_SCHEMA',), ('default',), ('information_schema',), ('superficie_bd',), ('system',)]


In [6]:
# Read CSV files
csv_path_0 = '/usr/workspace/media/00_superficie_forestal.csv'
data_0 = pd.read_csv(csv_path_0)

In [7]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 22 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   entidad_federativa                                        32 non-null     object 
 1   poblacion                                                 32 non-null     int64  
 2   superficie_total                                          32 non-null     float64
 3   superficie_forestal                                       32 non-null     float64
 4   superficie_no_forestal                                    32 non-null     float64
 5   superficie_con_arbolado                                   32 non-null     float64
 6   area_cubierta_por_bosque                                  32 non-null     float64
 7   area_cubierta_por_selva                                   32 non-null     float64
 8   area_cubierta_por_mang

In [8]:
data_tuples_0 = [tuple(row) for row in data_0.values]
#data_tuples_0

In [9]:
# CREATE TABLE statement
create_table_0 = """
CREATE TABLE IF NOT EXISTS superficie_bd.superficie_forestal (
    entidad_federativa String,
    poblacion UInt32,
    superficie_total Float64,
    superficie_forestal Float64,
    superficie_no_forestal Float64,
    superficie_con_arbolado Float64,
    area_cubierta_por_bosque Float64,
    area_cubierta_por_selva Float64,
    area_cubierta_por_manglar Float64,
    superficie_cubierta_por_otras_areas_arboladas Float64,
    area_cubierta_por_matorral_xerofilo Float64,
    area_cubierta_por_otras_areas_forestales Float64,
    superficie_destinada_a_actividades_agricolas_de_humedad Float64,
    superficie_destinada_a_actividades_agricolas_de_riego Float64,
    superficie_destinada_a_actividades_agricolas_de_temporal Float64,
    superficie_de_cuerpos_de_agua Float64,
    superficie_destinada_a_actividades_acuicolas Float64,
    area_de_pastizales_cultivados Float64,
    area_de_pastizales_inducidos Float64,
    superficie_sin_vegetacion_visible Float64,
    superficie_desprovista_de_vegetacion Float64,
    superficie_ocupada_por_asentamientos_humanos Float64
) ENGINE = MergeTree()
ORDER BY entidad_federativa
"""

In [10]:
# Execute the CREATE TABLE statement
try:
    client.command(create_table_0)
    print("Table 'superficie_forestal' created successfully!")
except Exception as e:
    print(f"Error creating table: {e}")

Table 'superficie_forestal' created successfully!


In [11]:
# Insert the DataFrame data
try:
    client.insert('superficie_bd.superficie_forestal', data_tuples_0)
    print(f"Successfully inserted {len(data_0)} rows into superficie_forestal table")
except Exception as e:
    print(f"Error inserting data: {e}")

Successfully inserted 32 rows into superficie_forestal table


In [12]:
result = client.query('''SELECT entidad_federativa, poblacion, area_cubierta_por_bosque
                        FROM superficie_bd.superficie_forestal
                        ORDER BY area_cubierta_por_bosque DESC''')

print(result.column_names)
result.result_rows

('entidad_federativa', 'poblacion', 'area_cubierta_por_bosque')


[('Chihuahua', 3741869, 7007289.07),
 ('Durango', 1832650, 5193060.023),
 ('Oaxaca', 4132148, 3714255.441),
 ('Jalisco', 8348151, 2366019.611),
 ('Guerrero', 3540685, 2306850.12),
 ('Sonora', 2944840, 1973339.456),
 ('Michoacan', 4748846, 1652546.271),
 ('Chiapas', 5543828, 1591903.464),
 ('Zacatecas', 1622138, 1084248.013),
 ('Nayarit', 1235456, 936335.3359),
 ('Sinaloa', 3026943, 870592.6537),
 ('Coahuila', 3146771, 836629.6729),
 ('Nuevo Leon', 5784442, 687653.6271),
 ('Estado de Mexico', 16992418, 623624.7159),
 ('Puebla', 6583278, 620601.0796),
 ('Tamaulipas', 3527735, 607746.5879),
 ('San Luis Potosi', 2822255, 569355.949),
 ('Hidalgo', 3082841, 545670.3645),
 ('Guanajuato', 6166934, 477178.7525),
 ('Veracruz', 8062579, 315966.6948),
 ('Queretaro', 2368467, 280065.3956),
 ('Baja California Norte', 8769020, 176522.3164),
 ('Aguascalientes', 1425607, 101575.2101),
 ('Tlaxcala', 1342977, 66710.85055),
 ('Baja California Sur', 798447, 58469.30527),
 ('Morelos', 1971520, 54185.63411),

In [13]:
result = client.query('''SELECT entidad_federativa, area_cubierta_por_selva
                        FROM superficie_bd.superficie_forestal
                        ORDER BY area_cubierta_por_selva DESC
                        LIMIT 1''')

print(result.column_names)
result.result_rows

('entidad_federativa', 'area_cubierta_por_selva')


[('Campeche', 3782854.848)]

In [14]:
result = client.query('''SELECT
                            entidad_federativa,
                            area_cubierta_por_bosque/poblacion as indice_bosque_poblacion
                        FROM superficie_bd.superficie_forestal
                        ORDER BY
                            area_cubierta_por_bosque/poblacion DESC
                        ''')

print(result.column_names)
result.result_rows

('entidad_federativa', 'indice_bosque_poblacion')


[('Durango', 2.833634367173219),
 ('Chihuahua', 1.8726708684884479),
 ('Oaxaca', 0.898867959472894),
 ('Nayarit', 0.7578864288975082),
 ('Sonora', 0.6701007375612936),
 ('Zacatecas', 0.668406765022458),
 ('Guerrero', 0.651526504052182),
 ('Michoacan', 0.3479890211221842),
 ('Sinaloa', 0.28761448553871016),
 ('Chiapas', 0.28714878311520486),
 ('Jalisco', 0.28341840139211666),
 ('Coahuila', 0.26586925864640293),
 ('San Luis Potosi', 0.2017379538702208),
 ('Hidalgo', 0.17700243525371565),
 ('Tamaulipas', 0.17227671236643344),
 ('Nuevo Leon', 0.11887985515283929),
 ('Queretaro', 0.11824753969550768),
 ('Puebla', 0.09426931075977651),
 ('Guanajuato', 0.07737698384643),
 ('Colima', 0.07398733325950141),
 ('Baja California Sur', 0.0732287869702059),
 ('Aguascalientes', 0.0712504989804343),
 ('Tlaxcala', 0.04967385930660019),
 ('Veracruz', 0.03918928357787254),
 ('Estado de Mexico', 0.03670017509573976),
 ('Morelos', 0.027484191948344425),
 ('Baja California Norte', 0.02013022166673129),
 ('Ca

In [15]:
result = client.query('''SELECT 
                            SUM(superficie_destinada_a_actividades_agricolas_de_humedad)
                            +SUM(superficie_destinada_a_actividades_agricolas_de_riego)
                            +SUM(superficie_destinada_a_actividades_agricolas_de_temporal)
                            +SUM(superficie_de_cuerpos_de_agua)
                            +SUM(superficie_destinada_a_actividades_acuicolas)
                            +SUM(area_de_pastizales_cultivados)
                            +SUM(area_de_pastizales_inducidos)
                            +SUM(superficie_sin_vegetacion_visible)
                            +SUM(superficie_desprovista_de_vegetacion)
                            +SUM(superficie_ocupada_por_asentamientos_humanos)
                        AS superficie_no_forestal
                        FROM
                          superficie_bd.superficie_forestal
                        WHERE
                            entidad_federativa = 'Estado de Mexico'
                      ''')

print(result.column_names)
result.result_rows

('superficie_no_forestal',)


[(1447249.8808434,)]